<h1 align='center'> COMP2420/COMP6420 - Introduction to Data Management,<br/> Analysis and Security</h1>

<h1 align='center'> Assignment - 2</h1>

-----
<br/>

## Grading

|**Maximum Marks**         |  **100**
|--------------------------|--------
|  **Weight**              |  **20% of the Total Course Grade**
|  **Submission deadline** |  **7:00PM, Friday, May 24**
|  **Submission mode**     |  **Electronic, Using GitLab <br/> One submission per group**
|  **Estimated time**      |  **20 hours**
|  **Penalty**             |  **100% after the deadline**
  


## Submission

You need to submit the notebook `Assignment-2.ipynb` and any other additional files that you may have created / hyperlinked in this notebook, as part of your submission by pushing it to your forked GitLab repository. You need to add your group details below. Make sure your group works on and submits only have one fork of the assignment repository. 


### Note:

* It is strongly advised to read the whole assignment before attempting it and have at least a cursory glance at the dataset in order to gauge the requirements and understand what you need to do as a bigger picture.

* For answers requiring free form written text, use the designated cells denoted by `YOUR ANSWER HERE` -- double click on the cell to write inside them.

* For all coding questions please write your code after the comment `YOUR CODE HERE`.

* In the process of testing your code, you can insert more cells or use print statements for debugging, but when submitting your file remember to remove these cells and calls respectively.

* You will be marked on **correctness** and **readability** of your code, if your marker can't understand your code your marks may be deducted.

<span style="color:blue">
    
### Group Number :  2420_A2_Grp12

### Student IDs: u6555407, u6588836, u6611178

</span>

## Introduction



In [1]:
# IMPORTING FREQUENTLY USED PYTHON MODULES
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2
import os
plt.style.use('seaborn-notebook')
%matplotlib inline

# JUST TO MAKE SURE SOME WARNINGS ARE IGNORED 
import warnings
warnings.filterwarnings("ignore")

In [10]:
# IMPORT ANY OTHER REQUIRED MODULES IN THIS CELL
### TODO SECTION A: q4 & SECTION B Q1.1





***

## Section A - Database Management (15 Marks)

You have been given the following database containing 15 tables relating to **DVD Movie Rentals**. The data contained in these tables is as follows -

|  **Table**               |  **Data Description**      |
|--------------------------|----------------------------|
|  actor                   |  actors data including first name and last name     |
|  film                    |  films data such as title, release year, length, rating, etc      |
|  film_actor              |  stores the relationships between films and actors |
|  category                |  stores film’s categories data |
|  film_category           |  stores the relationships between films and categories | 
|  store                   |  store data including manager staff and address |
|  inventory               |  stores inventory data |
|  rental                  |  stores rental data |
|  payment                 |  stores customer's payments |
|  staff                   |  stores staff data |
|  customer                |  stores customer data |
|  address                 |  address data for staff and customers |
|  city                    |  stores city data |
|  country                 |  stores country data |

Visualizing the relations between these tables can be aided by looking at the below **E-R Diagram**.

<img src='./dvd_rental_er.png'>

Based on your understanding of the relationships between these tables, answer the following questions by writing SQL queries to get the required data rows from the database and display them as a **Pandas dataframe**. 

***
#### **NOTE**:
For the following questions, in the CSIT labs you will be able to connect directly to the database using the below code within the notebook. From outside of the CSIT labs, you will need to perform SQL queries in your terminal by using [partch](https://cs.anu.edu.au/docs/student-computing-environment/linuxlabs/remoteaccess/#connectingtopartch3). Once you have the correct query, you may just fill in the boxes below.

**Partch Instructions**
1. Connect to partch as per the above hyperlink
2. In your terminal, enter `psql` to access the sql database
3. Enter `\c dvdrental` to ensure you are accessing the assignment database (or simply specify dvdrental when connecting to the database server -- `psql dvdrental`).
4. (Sanity Check) Enter `SELECT * FROM actor;` . If you receive the first row to be "Penelope Guiness", you should be good to go !
***

In [3]:
# Connect using psycopg2
conn = psycopg2.connect(host="/var/run/postgresql", database="dvdrental")

# Activate connection cursor
curr = conn.cursor()

In [4]:
def execute_sql(query):
    # Select table and display
    curr.execute(query)

    # Fetches all the rows from the result of the query
    rows = curr.fetchall()
    
    # Gets the column names for the table
    colnames = [desc[0] for desc in curr.description]

    # Converts into readable pandas dataframe
    df_result = pd.DataFrame(rows, columns=colnames)
    return df_result

### 1. Write an SQL query to determine the total sales from the rentals of the film 'Affair Prejudice' from the rental store with `store_id = 2`. Your query should result in a single column called `Total Rental Cost` with the value of the total cost of all these rentals.
<span style= 'float: right;'><b>[3 marks]</b></span>

In [5]:
# YOUR QUERY HERE
query = str("SELECT SUM(p.amount) AS TOTAL_RENTAL_COST"
            +" FROM payment AS p,rental AS r,inventory AS i,film AS f"
            +" WHERE p.rental_id=r.rental_id"
            +" AND r.inventory_id=i.inventory_id"
            +" AND i.film_id=f.film_id"
            +" AND f.title LIKE 'Affair Prejudice'"
            +" AND i.store_id=2;")

execute_sql(query)

,total_rental_cost
0,33.91


### 2. Write an SQL query to retrieve the names of all customers who rented the film ‘Affair Prejudice’ from the store with `store_id = 1`. The result of your query should display each customer's first name and last name. 
<span style= 'float: right;'><b>[3 marks]</b></span>

In [6]:
# YOUR QUERY HERE
query = str("SELECT c.first_name,c.last_name"
    +" FROM customer AS c,rental AS r,inventory AS i,film AS f"
    +" WHERE c.customer_id=r.customer_id"
    +" AND r.inventory_id=i.inventory_id"
    +" AND i.film_id=f.film_id"
    +" AND f.title LIKE '%Affair Prejudice%'"
    +" AND c.store_id=1;")

execute_sql(query)

,first_name,last_name
0,Vivian,Ruiz
1,Nina,Soto
2,Jordan,Archuleta
3,Michael,Silverman
4,Eleanor,Hunt
5,Natalie,Meyer
6,Tommy,Collazo
7,Carmen,Owens
8,Deanna,Byrd
9,Jimmy,Schrader


### 3. Write an SQL query to retrieve the names of all those customers who have a total of more than 100 dollars in recorded payments. Your query's result should include each customer's first name, last name and customer ID. 
<span style= 'float: right;'><b>[4 marks]</b></span>

In [7]:
# YOUR QUERY HERE
query = str("SELECT c.first_name,c.last_name,c.customer_id" 
            +" FROM customer AS c, payment AS p"
            +" WHERE c.customer_id=P.customer_id"
            +" GROUP BY c.customer_id HAVING SUM(p.amount)>100;")

execute_sql(query)

,first_name,last_name,customer_id
0,Wanda,Patterson,87
1,Dan,Paine,477
2,Priscilla,Lowe,273
3,Guy,Brownlee,550
4,Alice,Stewart,51
5,Yolanda,Weaver,190
6,Kyle,Spurlock,424
7,Nathan,Runyon,406
8,June,Carroll,176
9,Morris,Mccarter,576


### 4. Write an SQL query to retrieve the names of all customers who have rented the movie ‘Angels Life’ from *both* stores with `store_id = 1` and `store_id = 2`. Your query's result should include each customer's first name, last name and customer ID.
<span style= 'float: right;'><b>[5 marks]</b></span>

In [8]:
# YOUR QUERY HERE

query = str("SELECT c.customer_id,c.first_name,c.last_name"
    +" FROM customer AS c,rental AS r,inventory AS i,film AS f"
    +" WHERE c.customer_id=r.customer_id"
    +" AND c.store_id IN ('1','2')"
    +" AND r.inventory_id=i.inventory_id"
    +" AND i.film_id=f.film_id"
    +" AND f.title='Angels Life';")
        
execute_sql(query)

,customer_id,first_name,last_name
0,265,Jennie,Terry
1,497,Gilbert,Sledge
2,158,Veronica,Stone
3,377,Howard,Fortner
4,163,Cathy,Spencer
5,141,Debbie,Reyes
6,558,Jimmie,Eggleston
7,327,Larry,Thrasher
8,147,Joanne,Robertson
9,225,Arlene,Harvey


## Section B - Data Acquisition (20 Marks)

In this section, you need to read and import the data from the database into Python and make it ready to be used by your machine learning algorithms in the next section. **Your task here is to load the required columns from the various appropriate tables in the database to fuel the data required to train your machine learning models in the next section.** You can load the required data into one or more Pandas dataframes, to suit the the needs of different Machine Learning models in Section C. 

If you fail to do so, or an error in the previous section is preventing you from doing so, we can provide you with a **CSV of the required data**. If you choose to use this CSV, you will not receive any marks for this section, but you'll be able to do the following section without having to spend time on this section.  

<span style='color:red;'><b>Note:</b> While you are provided the .csv files to use for development of the later questions at home, you must submit a copy of the code that can read the database to receive marks for this section.</span>
<span style= 'float: right;'><b>[20 marks]</b></span>

In [12]:
# YOUR CODE HERE


query=str("SELECT * FROM customer c"
+" JOIN rental r"
+" ON c.customer_id=r.rental_id"
+" JOIN store s" 
+" ON c.store_id=s.store_id"
+" JOIN payment p"
+ " ON p.customer_id=c.customer_id"
+" JOIN address a"
+" ON c.address_id=a.address_id"
+" JOIN city"
+" ON city.city_id=a.city_id"
+" JOIN country"
+" ON country.country_id=city.country_id"
+" JOIN inventory i"
+" ON i.inventory_id=r.inventory_id"
+" JOIN staff"
+ " ON staff.store_id=s.store_id"
+" JOIN film"
+" ON i.inventory_id=film.film_id"
+" JOIN language l"
+" ON l.language_id=film.language_id"
+" JOIN film_actor fa"
+" ON fa.film_id=film.film_id"
+" JOIN actor"
+" ON actor.actor_id=fa.actor_id"
+" JOIN film_category fc"
+" ON fc.film_id=film.film_id"
+" JOIN category"
+" ON category.category_id=fc.category_id;"
)
data1 = execute_sql(query)
data1



,customer_id,store_id,first_name,last_name,email,address_id,activebool,create_date,last_update,active,...,actor_id,first_name,last_name,last_update,film_id,category_id,last_update,category_id,name,last_update
0,361,2,Lawrence,Lawton,lawrence.lawton@sakilacustomer.org,366,True,2006-02-14,2013-05-26 14:49:45.738,1,...,21,Kirsten,Paltrow,2013-05-26 14:47:57.620,6,9,2006-02-15 10:07:09,9,Foreign,2006-02-15 09:46:27
1,361,2,Lawrence,Lawton,lawrence.lawton@sakilacustomer.org,366,True,2006-02-14,2013-05-26 14:49:45.738,1,...,21,Kirsten,Paltrow,2013-05-26 14:47:57.620,6,9,2006-02-15 10:07:09,9,Foreign,2006-02-15 09:46:27
2,361,2,Lawrence,Lawton,lawrence.lawton@sakilacustomer.org,366,True,2006-02-14,2013-05-26 14:49:45.738,1,...,21,Kirsten,Paltrow,2013-05-26 14:47:57.620,6,9,2006-02-15 10:07:09,9,Foreign,2006-02-15 09:46:27
3,361,2,Lawrence,Lawton,lawrence.lawton@sakilacustomer.org,366,True,2006-02-14,2013-05-26 14:49:45.738,1,...,21,Kirsten,Paltrow,2013-05-26 14:47:57.620,6,9,2006-02-15 10:07:09,9,Foreign,2006-02-15 09:46:27
4,361,2,Lawrence,Lawton,lawrence.lawton@sakilacustomer.org,366,True,2006-02-14,2013-05-26 14:49:45.738,1,...,21,Kirsten,Paltrow,2013-05-26 14:47:57.620,6,9,2006-02-15 10:07:09,9,Foreign,2006-02-15 09:46:27
5,361,2,Lawrence,Lawton,lawrence.lawton@sakilacustomer.org,366,True,2006-02-14,2013-05-26 14:49:45.738,1,...,21,Kirsten,Paltrow,2013-05-26 14:47:57.620,6,9,2006-02-15 10:07:09,9,Foreign,2006-02-15 09:46:27
6,361,2,Lawrence,Lawton,lawrence.lawton@sakilacustomer.org,366,True,2006-02-14,2013-05-26 14:49:45.738,1,...,21,Kirsten,Paltrow,2013-05-26 14:47:57.620,6,9,2006-02-15 10:07:09,9,Foreign,2006-02-15 09:46:27
7,361,2,Lawrence,Lawton,lawrence.lawton@sakilacustomer.org,366,True,2006-02-14,2013-05-26 14:49:45.738,1,...,21,Kirsten,Paltrow,2013-05-26 14:47:57.620,6,9,2006-02-15 10:07:09,9,Foreign,2006-02-15 09:46:27
8,361,2,Lawrence,Lawton,lawrence.lawton@sakilacustomer.org,366,True,2006-02-14,2013-05-26 14:49:45.738,1,...,21,Kirsten,Paltrow,2013-05-26 14:47:57.620,6,9,2006-02-15 10:07:09,9,Foreign,2006-02-15 09:46:27
9,361,2,Lawrence,Lawton,lawrence.lawton@sakilacustomer.org,366,True,2006-02-14,2013-05-26 14:49:45.738,1,...,21,Kirsten,Paltrow,2013-05-26 14:47:57.620,6,9,2006-02-15 10:07:09,9,Foreign,2006-02-15 09:46:27


***

In [13]:
data1.columns

Index(['customer_id', 'store_id', 'first_name', 'last_name', 'email',
       'address_id', 'activebool', 'create_date', 'last_update', 'active',
       'rental_id', 'rental_date', 'inventory_id', 'customer_id',
       'return_date', 'staff_id', 'last_update', 'store_id',
       'manager_staff_id', 'address_id', 'last_update', 'payment_id',
       'customer_id', 'staff_id', 'rental_id', 'amount', 'payment_date',
       'address_id', 'address', 'address2', 'district', 'city_id',
       'postal_code', 'phone', 'last_update', 'city_id', 'city', 'country_id',
       'last_update', 'country_id', 'country', 'last_update', 'inventory_id',
       'film_id', 'store_id', 'last_update', 'staff_id', 'first_name',
       'last_name', 'address_id', 'email', 'store_id', 'active', 'username',
       'password', 'last_update', 'picture', 'film_id', 'title', 'description',
       'release_year', 'language_id', 'rental_duration', 'rental_rate',
       'length', 'replacement_cost', 'rating', 'last_update',

In [20]:
data1.iloc[250:255]

,customer_id,store_id,first_name,last_name,email,address_id,activebool,create_date,last_update,active,...,actor_id,first_name,last_name,last_update,film_id,category_id,last_update,category_id,name,last_update
250,465,1,Floyd,Gandy,floyd.gandy@sakilacustomer.org,470,True,2006-02-14,2013-05-26 14:49:45.738,1,...,136,Ed,Mansfield,2013-05-26 14:47:57.620,20,12,2006-02-15 10:07:09,12,Music,2006-02-15 09:46:27
251,465,1,Floyd,Gandy,floyd.gandy@sakilacustomer.org,470,True,2006-02-14,2013-05-26 14:49:45.738,1,...,139,Ewan,Gooding,2013-05-26 14:47:57.620,20,12,2006-02-15 10:07:09,12,Music,2006-02-15 09:46:27
252,465,1,Floyd,Gandy,floyd.gandy@sakilacustomer.org,470,True,2006-02-14,2013-05-26 14:49:45.738,1,...,139,Ewan,Gooding,2013-05-26 14:47:57.620,20,12,2006-02-15 10:07:09,12,Music,2006-02-15 09:46:27
253,465,1,Floyd,Gandy,floyd.gandy@sakilacustomer.org,470,True,2006-02-14,2013-05-26 14:49:45.738,1,...,139,Ewan,Gooding,2013-05-26 14:47:57.620,20,12,2006-02-15 10:07:09,12,Music,2006-02-15 09:46:27
254,465,1,Floyd,Gandy,floyd.gandy@sakilacustomer.org,470,True,2006-02-14,2013-05-26 14:49:45.738,1,...,139,Ewan,Gooding,2013-05-26 14:47:57.620,20,12,2006-02-15 10:07:09,12,Music,2006-02-15 09:46:27


## Section C - Machine Learning (55 Marks)
<span style='color:purple;'><b>Note:</b> We have provided the .csv outputs for the files to use in these questions if you are working on the assignment at home. Please note if any of these questions refer to those csvs and not the data from Section B in your submitted assignment, you will receive no marks for Section B.</span>

In [53]:
## Import csv files for home development here as necessary. 
df_actor=pd.read_csv("data/dvd_rental/actor.csv")
df_address=pd.read_csv("data/dvd_rental/address.csv")
df_category=pd.read_csv("data/dvd_rental/category.csv")
df_city=pd.read_csv("data/dvd_rental/city.csv")
df_country=pd.read_csv("data/dvd_rental/country.csv")
df_film=pd.read_csv("data/dvd_rental/film.csv")
df_film_actor=pd.read_csv("data/dvd_rental/film_actor.csv")
df_film_category=pd.read_csv("data/dvd_rental/film_category.csv")
df_inventory=pd.read_csv("data/dvd_rental/inventory.csv")
df_language=pd.read_csv("data/dvd_rental/language.csv")
df_payment=pd.read_csv("data/dvd_rental/payment.csv")
df_rental=pd.read_csv("data/dvd_rental/rental.csv")
df_staff=pd.read_csv("data/dvd_rental/staff.csv")
df_store=pd.read_csv("data/dvd_rental/store.csv")

In [54]:
def feature_combinations(dataframe,dependent_variable):
    '''Helper function for producing all possible combinations of independent variables 
       based on the given dataframe & the dependent_variable
       @param:dataframe - The primary DataFrame in which your data resides.
       @return:output - A list of list which has all possible combinations.'''
    output=[]
    independent_variables=[]
    for var in dataframe.columns:
            if var != dependent_variable:
                independent_variables.append(var)
    for i in range(len(independent_variables)):
           for combination in list(combinations(independent_variables,i+1)):
                output.append(combination)
    return output        

### Part 1 - Clustering (25 Marks)

### Part 1.1 - Clustering Short Answers (8 Marks)

#### 1. Can a Decision Tree be used to perform clustering? If so, explain how. If not, provide a counterexample showing how it is not suitable for the function.
<span style= 'float: right;'><b>[4 marks]</b></span>

#### <span style="color:blue"> Your answer here: </span>
- For a given dataset,decision trees can be used to perform only classification,not clustering [(Both are similar,but different)](https://techdifferences.com/difference-between-classification-and-clustering.html) .Decision Trees are a form of Supervised learning and they classify instances based on a known/predefined labels.Decision Trees will use Heuristics(attribute selection measures) such as Information Gain,Gain Ratio or Gini index to decide the decision nodes of the tree.
- Counter - Example:


#### 2. Will a K-means Clustering Algorithm generate the same results each time? Provide examples on how this may or may not be the case.
<span style= 'float: right;'><b>[4 marks]</b></span>

#### <span style="color:blue"> Your answer here: </span>

(Assumed that the centroids are randomly assigned and the number of the centroids are the same each time)

The results are usually different. But, it depends on the data and the centroids number. For example, if the data has obvious clusters, and the number of centroids are equal to its clusters. K-means Clustering Algorithm will generate the same result each time no matter where the initial centroids are. For other kinds of data, the result usually depends on the initial position of the centroids.

### Part 1.2 - K-Means Clustering Implementation (17 Marks)

Clustering helps visualise a dataset based on attributes considered important to the data scientist and/or reader.  From the dataset acquired after completing **Section - B**, implement a `K-Means clustering algorithm` to help you cluster the dataset of customers on the basis of the movies they have rented from the DVD rental stores. Various attributes related to the movies such `rating`, `year`, `rental_rate` and `year` may be useful for this exercise. Another interesting attribute that you can look at is the `fulltext` of the movie. In terms of the customer details, you can look at which movies a customer has rented, and the total number of movies rented by a customer. After you've prepared your learning model, plot a **visualization** showing the different clusters. If you have used more than 2 features for your clustering, you are still expected to provide a visualization by reducing the dimensions into a 2D graph.

It's upto you to decide how many clusters you would like to incorporate in your model. You are expected to justify all aspects of your implementation including the reasoning behind the choice of **the number of clusters** and **number of iterations** in your model. 

<span style='color:red;'><b>Note:</b> You are only allowed to use packages that are within the Anaconda distribution.</span>
<span style= 'float: right;'><b>[15 marks total: 10 marks model, 5 marks justification & commentary]</b></span>

In [ ]:
# YOUR CODE HERE






#### <span style="color:blue"> Your justifications & commentary here: </span>

**

#### What do your resulting clusters represent?  Explain the distinguishing characteristics of each cluster. 
<span style= 'float: right;'><b>[2 marks]</b></span>

#### <span style="color:blue"> Your answer here: </span>

**

***

### Part 2 - Classification (30 Marks)

The rental company has decided that they want to use simple machine learning to allocate price tags to their movies. The tags are as follows:


|  **Classification**      |  **Requirements**        |
|--------------------------|------------------------- |
|  Cheap                   |  Rental Rate is 0.99   |
|  Moderate                |  Rental Rate is 2.99   |
|  Expensive               |  Rental Rate is 4.99  |


**Your task is to implement a `Classification Algorithm` (such as K-Nearest Neighbours) that can predict the `Price Label` of a movie**. You are required to perform the following tasks:

1. Create useable dataset/s by manually determining the 'truth values' for existing data (where the rating is within the classification system defined above)
2. Implement an algorithm that can predict the classification as per the above classifications.
3. Perform independent testing of the model and provide statistical metrics outlining the performance of your model. Splitting the dataset into testing and training subsets will assist with this.

You are welcome to use any features within the dataset, except the `Rental Rate` of the film. Various attributes relating to a movie in the tables `rating`, `movie`, `film_actor`, `actor` and `film_category` can be helpful while making the algorithm. If required, you can also look to make new **compound attributes** that may be helpful in increasing the accuracy of your model.
You are expected to justify all aspects of your answer including the features used, the metrics provided and validation system employed. Provide commentary on the strengths and potential pitfalls of the model.

<span style='color:red;'><b>Note:</b> You are only allowed to use packages that are within the Anaconda distribution. This means packages such as Keras, Tensorflow etc are not available for use.</span> 
<span style= 'float: right;'><b>[25 marks total: 18 marks model, 7 marks justification & commentary]</b></span>

In [ ]:
# YOUR CODE HERE



#### <span style="color:blue"> Your justifications & commentary here: </span>

**

#### Would you be able to get a better result, if you had used Clustering as a pre-processing step before Classfication? Justify your answer. 
<span style= 'float: right;'><b>[5 marks]</b></span>

#### <span style="color:blue"> Your answer here: </span>

**

***

## Section D - Decision Trees (10 Marks)

The following is a small synthetic data set about the weather conditions.  We are
going to try and use decision trees to predict whether it will rain or not on the given day.


|Temperature| Cloudy| UV Index| Humidity| Rain
|---:|--:|--:|--:|--:|
|25|No| Low| Low| No 
|29|No| Low| High| No
|26|No| Low| Medium| No
|26|No| Medium| Medium| No
|27|No| Medium| High| No
|28|No| High | High| No
|25|No| High |Low| No
|29|Yes| Low |Low| Yes
|28|No| Medium| High| Yes
|28|Yes| Medium| High| Yes
|26|No| Low |Low| Yes
|27|Yes| Low |High| Yes

**Note:**
* You can treat temperature as a continuous variable and split it on a range of values (to convert it to a categorical variable, for example).
* Attribute selection in the tree uses information gain.
* You can use LaTeX and/or markdown to format your equations.

#### 1. What is the initial entropy of Cloudy?

<span style= 'float: right;'><b>[3 marks]</b></span>

<span style="color:blue">
    
#### YOUR ANSWER HERE


</span>

#### 2. Which attribute would the decision-tree building algorithm choose at the root of the tree?

<span style= 'float: right;'><b>[3 marks]</b></span>

<span style="color:blue">
    
#### YOUR ANSWER HERE


</span>

#### 3. Calculate and specify the information gain of the attribute you chose to split on in the previous question

<span style= 'float: right;'><b>[4 marks]</b></span>

<span style="color:blue">
    
#### YOUR ANSWER HERE


</span>